# A pizza order bot

This sample notebook demonstrates  creation of a simple chatbot that uses an FFModel inference endpoint which in turn calls OpenAPI chat completions api.   The goal is to demonstrate interactions with the endpoint in a scenario that requires history being sent with the prompt.  The endpoint is designed to return json so successful results will look "funny" in the chat.  

In [ ]:
from ffmodel.core.inference_endpoint import InferenceEndpoint

environment_config_path = "~/.chatbot.env"

#solution_config_path = "../solution/configs/00_baseline.yaml"
#solution_config_path = "../solution/configs/01_with_static_context_01.yaml"
#solution_config_path = "../solution/configs/02_with_fewshots_01 _min.yaml"
#solution_config_path = "../solution/configs/03_with_fewshots_01.yaml"
solution_config_path = "../solution/configs/05_with_fewshots_02_min.yaml"
#solution_config_path = "../solution/configs/04_with_fewshots_02.yaml"

endpoint = InferenceEndpoint(solution_config_path, environment_config_path)

## 1.0 Define Request to OpenAI API
For these labs we are using the Azure OpenAI Chat Completion API, this is different than the Completions API.  Information about its usage can be found here:
<https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions>
The chat completions API takes a collection of messages to provide history and context of chat to the model.  This function takes a chat request object that ffmodel inference endpoint can use to create a prompt.

In [2]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(chat_request):
    response = endpoint.execute(chat_request)
    return response.model_output.completions[0]



## 2.0 The conversation

In [ ]:
import json
import typing
import uuid

from pydantic import BaseModel
from fastapi.encoders import jsonable_encoder

class CompletionPair(BaseModel):
    user_nl:str
    completion:object

class UserSession(BaseModel):
    session_id:str
    prior_responses:list[CompletionPair]
    sequence: typing.Optional[int]


class ChatRequest(BaseModel):
    user_nl:str
    session:UserSession

# This is the first user message that will be sent to the model. Feel free to update this.
max_response_tokens = 500
token_limit=2048
name = input('Enter Your Name: ')
print ('Welcome to pizzaai.  What can I get for you?')
conversation=[]
session_id=str(uuid.uuid4())
message_count=0
while True:
    request = input(name+':')
    current_message={request}
    chat_request=ChatRequest(user_nl=request,session=UserSession(session_id=session_id,sequence=message_count,prior_responses=conversation))
    if request=="Bye" or request=='bye':
        print('Bot: Bye')
        break
    else:
        payload=json.dumps(jsonable_encoder(chat_request))
        response = send_message(payload)
        interaction = CompletionPair(user_nl=request,completion=response)
        conversation.append(interaction)
        message_count+=1
        print('Bot: ', response)